# Atlas mapping

In [ ]:
# In the remapping acronyms are not lateralised, so must used either x value or atlas ids to differentiate

In [ ]:
from ibllib.atlas import AllenAtlas
ba = AllenAtlas()

## Available Mappings
Three mappings are currently available within the IBL, these are:

1. Allen Atlas - total of xx annotation regions provided by Allen Atlas
2. Beryl Atlas - total of xx annotation regions
3. Cosmos Atlas - total of 8 annotation regions

In [ ]:
# create figure
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 3, figsize=(15, 18))

# plot coronal slice at ap = -2000 um
ap = -2000 / 1e6
# Allen mapping
ba.plot_cslice(ap, volume='annotation', mapping='Allen', ax=axs[0])
_ = axs[0].set_title('Allen')
# Beryl mapping
ba.plot_cslice(ap, volume='annotation', mapping='Beryl', ax=axs[1])
_ = axs[1].set_title('Beryl')
# Cosmos mapping
ba.plot_cslice(ap, volume='annotation', mapping='Cosmos', ax=axs[2])
_ = axs[2].set_title('Cosmos')

## Some nomecluture
The brain_atlas comprises a 3D array that represents the brain volume. Each element in the array is and each element has a brain region assigned to it

## Understanding the mappings
The mappings store the highest level annotation (or parent node) that should be considered. Any regions that are children of these nodes are assigned the same annotation as their parent. 

For example, consider the region with the acronym **MDm** (Mediodorsal nucleus of the thalamus, medial part). Firstly, to navigate ourselves, we can find the acronyms of all the ancestors to this region,

In [ ]:
# First find the atlas_id associated with acronym MDm
atlas_id = ba.regions.acronym2atlasID('MDm')
# Then find the acronyms of the ancestors of this region
ba.regions.ancestors(ids=atlas_id)['acronym']

We can then take a look at what acronym this region will assigned under the different mappings. Under the Allen mapping we expect it to be assigned the same acronym as this is the lowest level region parcelation that we use.

In [ ]:
from iblutil.numerical import ismember
import numpy as np
ra, inds = ismember(ba.regions.acronym, np.array(['MDm', 'TH', 'DG']))
np.where(_)
help(ismember)
print(inds)
print(np.where(ra))
ba.regions.acronym

s = np.argsort(inds)
bla = np.where(ra)[0][s]
bla.reshape(3, 2)
1801 - 474

np.array('MDm')

In [ ]:
import numpy as np
la = np.array()
la

In [ ]:
ba.regions.acronym

In [ ]:
acronym2acronym(['MDm'], mapping='Allen')

Under the Beryl mapping, **MDm** is given the acronym of it's parent, **MD**

In [ ]:
acronym2acronym('MDm', mapping='Beryl')

Under the Cosmos mapping, it is assigned to the region **TH**

In [ ]:
acronym2acronym('MDm', mapping='Cosmos')

Therefore any clusters that are assigned an acronym **MDm** in the Allen mapping, will instead be assigned to the region **MD** in the Beryl mapping and **TH** in the Cosmos mapping

## Lateralisation
Lateralised versions of each of the three mappings are also available. This allow regions on the left and right hemispheres of the brain to be differentiated. 

The convention used is that regions in the left hemisphere have negative atlas ids whereas those on the right hemisphere have positive atlas ids.

For the non lateralised mappings the atlas id is always positive regardless of whether the region lies in the left or right hemisphere. Aggregating values over regions could result, therefore, in values from different hemispheres being considered together.

One thing to be aware of, is that while lateralised mappings return distinct atlas ids for the left and right hemispheres, the acronyms returned are not lateralised. 

For example consider findng the atlas id when mapping the acronym **MDm** onto the Beryl atlas. When specifying the left hemisphere, the returned atlas id is negative

In [ ]:
# Left hemisphere gives negative atlas id
acronym2atlasID('MDm', mapping='Beryl', hemisphere='left')

When specifying the right hemisphere, the returned atlas id is positive

In [ ]:
# Left hemisphere gives negative atlas id
acronym2atlasID('MDm', mapping='Beryl', hemisphere='right')

However, regardless of whether we specify left or right hemisphere, the returned acronym is always the same

In [ ]:
acronym2acronym('MDm', mapping='Beryl', hemisphere='right')

In [ ]:
acronym2acronym('MDm', mapping='Beryl', hemisphere='left')

In [ ]:
#acronym2atlas_id(acronym, mapping)
#acronym2index(acronym, mapping)
#atlasid2acronym()
br = ba.regions
def acronym2atlasID(acronym, mapping='Allen', hemisphere='both'):
    
    index = np.where(br.acronym == acronym)[0]
    br.id[br.mappings[mapping]][index]
    #idx = np.where(br.acronym[br.mappings[mapping]] == acronym)
    #print(idx)
    return br.id[br.mappings[mapping]][index]

def acronym2acronym(acronym, mapping='Allen', hemisphere='both'):
    # TODO account for cases where no mapping
    index = np.where(br.acronym == acronym)[0]
    return br.acronym[br.mappings[mapping]][index]

def acronym2index(acronym, mapping='Allen', hemisphere='both'):
    return np.where(br.acronym == acronym)[0]

In [ ]:
acronym2acronym('MDm', mapping='Beryl')

In [ ]:
#We have list of brain regioty
import pandas as pd
ba.regions.__dict__

In [ ]:
# brain volume
to interpret must use atlas_ids

## How to map your data

### Mapping from mlapdv coordinates